# Before you start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [95]:
# import numpy and pandas
import numpy as np
import pandas as pd
import math
from scipy.stats import t, ttest_1samp, ttest_rel, ttest_ind
from scipy.stats import norm
from scipy import stats

# Challenge 1 - Exploring the Data

In this challenge, we will examine all salaries of employees of the City of Chicago. We will start by loading the dataset and examining its contents. Please, load the data using Ironhack's database (db: employees, table: employees_advanced).

In [33]:
# Your code here:
data=pd.read_csv('/home/davidgozalo/Ironhack/dataptmad0420/module-2/hypothesis-testing/hypothesis-testing.csv')

Examine the `salaries` dataset using the `head` function below.

In [34]:
# Your code here:
data.head()



,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,NaN
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86


We see from looking at the `head` function that there is quite a bit of missing data. Let's examine how much missing data is in each column. Produce this output in the cell below

In [35]:
# Your code here:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33183 entries, 0 to 33182
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Name               33183 non-null  object 
 1   Job Titles         33183 non-null  object 
 2   Department         33183 non-null  object 
 3   Full or Part-Time  33183 non-null  object 
 4   Salary or Hourly   33183 non-null  object 
 5   Typical Hours      8022 non-null   float64
 6   Annual Salary      25161 non-null  float64
 7   Hourly Rate        8022 non-null   float64
dtypes: float64(3), object(5)
memory usage: 2.0+ MB


In [36]:
null_df=data.isnull()

In [37]:
null_df=data.notnull()

In [38]:
data.columns

Index(['Name', 'Job Titles', 'Department', 'Full or Part-Time',
       'Salary or Hourly', 'Typical Hours', 'Annual Salary', 'Hourly Rate'],
      dtype='object')

In [39]:
data_null = [data['Name'].isnull() | data['Job Titles'].isnull() | data['Department'].isnull() | data['Full or Part-Time'].isnull() | data['Salary or Hourly'].isnull()| data['Typical Hours'].isnull()| data['Annual Salary'].isnull() | data['Hourly Rate'].isnull()]
display(data_null)

[0        True
 1        True
 2        True
 3        True
 4        True
          ... 
 33178    True
 33179    True
 33180    True
 33181    True
 33182    True
 Length: 33183, dtype: bool]

In [40]:
data_null = [data['Typical Hours'].isnull()]
display (data_null)


[0         True
 1         True
 2         True
 3         True
 4        False
          ...  
 33178     True
 33179     True
 33180     True
 33181     True
 33182     True
 Name: Typical Hours, Length: 33183, dtype: bool]

In [41]:
data["Typical Hours"].count()

8022

In [42]:
data["Typical Hours"].value_counts()

40.0    5833
20.0    1901
10.0     184
35.0     104
Name: Typical Hours, dtype: int64

In [43]:
data_null_series = pd.Series(data_null) 

In [44]:
#depending on the function used to create the null_df (isnull() or notnull())
null_df["Typical Hours"].value_counts()

False    25161
True      8022
Name: Typical Hours, dtype: int64

Let's also look at the count of hourly vs. salaried employees. Write the code in the cell below

In [45]:
# Your code here:

data["Salary or Hourly"].value_counts()

Salary    25161
Hourly     8022
Name: Salary or Hourly, dtype: int64

What this information indicates is that the table contains information about two types of employees - salaried and hourly. Some columns apply only to one type of employee while other columns only apply to another kind. This is why there are so many missing values. Therefore, we will not do anything to handle the missing values.

There are different departments in the city. List all departments and the count of employees in each department.

In [46]:
# Your code here:
data_group = data[['Department','Name']].groupby('Department').count() 
display(data_group)

,Name
Department,
ADMIN HEARNG,39
ANIMAL CONTRL,81
AVIATION,1629
BOARD OF ELECTION,107
BOARD OF ETHICS,8
BUDGET & MGMT,46
BUILDINGS,269
BUSINESS AFFAIRS,171
CITY CLERK,84


In [47]:
data["Department"].value_counts()

POLICE                   13414
FIRE                      4641
STREETS & SAN             2198
OEMC                      2102
WATER MGMNT               1879
AVIATION                  1629
TRANSPORTN                1140
PUBLIC LIBRARY            1015
GENERAL SERVICES           980
FAMILY & SUPPORT           615
FINANCE                    560
HEALTH                     488
CITY COUNCIL               411
LAW                        407
BUILDINGS                  269
COMMUNITY DEVELOPMENT      207
BUSINESS AFFAIRS           171
COPA                       116
BOARD OF ELECTION          107
DoIT                        99
PROCUREMENT                 92
INSPECTOR GEN               87
MAYOR'S OFFICE              85
CITY CLERK                  84
ANIMAL CONTRL               81
HUMAN RESOURCES             79
CULTURAL AFFAIRS            65
BUDGET & MGMT               46
ADMIN HEARNG                39
DISABILITIES                28
TREASURER                   22
HUMAN RELATIONS             16
BOARD OF

# Challenge 2 - Hypothesis Tests

In this section of the lab, we will test whether the hourly wage of all hourly workers is significantly different from $30/hr. Import the correct one sample test function from scipy and perform the hypothesis test for a 95% two sided confidence interval.

In [48]:
# Your code here:

In [49]:
# I extract the relevant sample
data_hourly=data[data["Salary or Hourly"]=="Hourly"]

In [50]:
# H0:mu=30
mu=30

In [51]:
# I calculate the sample mean 
mu_sample=data_hourly["Hourly Rate"].mean()
print (mu_sample)

32.78855771628024


In [52]:
#I calculate the sample std
std_sample=data_hourly["Hourly Rate"].std()
print (std_sample)

12.112572684276799


In [53]:
# I calculate the sample size
n=data["Hourly Rate"].count()
print(n)

8022


In [54]:
# I calculate the t statistical
t_test=(mu_sample-mu)/(std_sample/(math.sqrt(n)))
print (t_test)

20.6198057854942


In [55]:
# It is a two sided test

In [56]:
p_value=2*(1-t.cdf(t_test,n-1))
print (p_value)

0.0


In [57]:
#If the extreme probability of this value is lower than the significance level,
#I can reject the null hypothesis 

cummulative_probability=t.cdf(t_test, n-1)
extreme_probability=(1-cummulative_probability)*2 #it is a "two side" case
print (extreme_probability)

0.0


In [58]:
# I can also use scipy

In [59]:
#I create a sample
sample=np.array(data_hourly["Hourly Rate"])
display (sample)

array([19.86, 46.1 , 35.6 , ..., 35.6 , 46.35, 48.85])

In [60]:
ttest_1samp(sample, 30)

Ttest_1sampResult(statistic=20.6198057854942, pvalue=4.3230240486229894e-92)

In [61]:
test_results = ttest_1samp(data_hourly["Hourly Rate"],30)
display(test_results.statistic)
display(test_results.pvalue)

20.6198057854942

4.3230240486229894e-92

In [62]:
#Since pvalue is a very close to zero number and it is lower than the significance level,0.05
#I can reject the null hypothesis
#This means that in a world where the null hipothesis is true (average hourly wage is 30$), 
#there’s a 0% chance we would see the average equal to 30$ due to random.
#We can assess, taking in accoun the data, that the hourly wage of all hourly workers 
#is significantly different from $30/hr. for a 95%confidence interval.

We are also curious about salaries in the police force. The chief of police in Chicago claimed in a press briefing that salaries this year are higher than last year's mean of $86000/year a year for all salaried employees. Test this one sided hypothesis using a 95% confidence interval.

Hint: A one tailed test has a p-value that is half of the two tailed p-value. If our hypothesis is greater than, then to reject, the test statistic must also be positive.

In [63]:
# Your code here:

In [64]:
# H0:mu<=86.000
mu=86000

In [65]:
# I extract the relevant sample
data_salary=data[data["Salary or Hourly"]=="Salary"]

In [66]:
# I calculate the sample mean 
mu_sample=data_salary["Annual Salary"].mean()
print (mu_sample)

86786.99979015143


In [67]:
# I calculate the sample std
std_sample=data_salary["Annual Salary"].std()
print (std_sample)

21041.35460162839


In [68]:
# I calculate the sample size
n=data["Annual Salary"].count()
print(n)

25161


In [69]:
# I calculate the t statistical
t_test=(mu_sample-mu)/(std_sample/(math.sqrt(n)))
print (t_test)

5.932870515690814


In [70]:
# It is a upper-tailed test

In [71]:
#If the extreme probability of this value is lower than the significance level, and the t statistical
#falls in the right side
#I can reject the null hypothesis 

cummulative_probability=t.cdf(t_test, n-1)
extreme_probability=(1-cummulative_probability) #it is a "right side" case
print (extreme_probability)

1.5078792658940188e-09


In [72]:
#I create a sample
sample=np.array(data_salary["Annual Salary"])
display (sample)

array([101442.,  94122., 101592., ...,  90024.,  93354., 115932.])

In [73]:
ttest_1samp(sample, 86000)

Ttest_1sampResult(statistic=5.932870515690814, pvalue=3.015758519916386e-09)

In [74]:
test_results = ttest_1samp(data_salary["Annual Salary"],86000)
display(test_results.statistic)
display(test_results.pvalue/2)

5.932870515690814

1.507879259958193e-09

In [75]:
#Since pvalue is a very close to zero number and it is lower than the significance level,and statistics
#falls in the right side ,I can reject the null hypothesis
#This means that in a world where the null hipothesis is true (salaries are lower or equal to 86.000$), 
#there’s a 0% chance we would see salaries are lower or equal to 86.000$ due to random
#We can assess, taking into acccount the data, that this year salaries are higher than last
#year for a 95% confidence interval.

Using the `crosstab` function, find the department that has the most hourly workers. 

In [76]:
# Your code here:

In [77]:
p=data["Department"]
r=data["Salary or Hourly"]

In [78]:
a=pd.crosstab(p, [r], rownames=['Department'], colnames=['Type'])

In [79]:
a.sort_values(by="Hourly", ascending=False).head()

Type,Hourly,Salary
Department,,
STREETS & SAN,1862,336
WATER MGMNT,1513,366
OEMC,1273,829
AVIATION,1082,547
GENERAL SERVICES,765,215


The workers from the department with the most hourly workers have complained that their hourly wage is less than $35/hour. Using a one sample t-test, test this one-sided hypothesis at the 95% confidence level.

In [192]:
# Your code here:

In [81]:
# H0:mu>=35
mu=35

In [82]:
# I extract the relevant sample
data_Streets_San=data[(data["Department"]=="STREETS & SAN")&(data["Salary or Hourly"]=="Hourly")]

In [83]:
# I calculate the sample mean 
mu_sample=data_Streets_San["Hourly Rate"].mean()
print (mu_sample)

33.72837808807734


In [84]:
# I calculate the sample std
std_sample=data_Streets_San["Hourly Rate"].std()
print (std_sample)

5.735241841459235


In [85]:
# I calculate the sample size
n=data_Streets_San["Hourly Rate"].count()
print(n)

1862


In [86]:
# I calculate the t statistical
t_test=(mu_sample-mu)/(std_sample/(math.sqrt(n)))
print (t_test)

-9.567447887848152


In [ ]:
#It is a lower_tailed test

In [ ]:
#If the extreme probability of this value is lower than the significance level, and the t statistical
#falls in the left side, I can reject the null hypothesis 

cummulative_probability=t.cdf(t_test, n-1)
extreme_probability=(1-cummulative_probability) #it is a "right side" case
print (extreme_probability)

In [88]:
#If the extreme probability of this value is lower than the significance level,
#I can reject the null hypothesis 

cummulative_probability=t.cdf(t_test, n-1)
extreme_probability=(cummulative_probability) #it is a "left side" case
print (extreme_probability)


1.6689265282353859e-21


In [89]:
sample=np.array(data_Streets_San["Hourly Rate"])
display (sample)

array([35.6 , 21.43, 35.6 , ..., 35.6 , 36.21, 22.12])

In [202]:
ttest_1samp(sample, 35)

Ttest_1sampResult(statistic=-9.567447887848152, pvalue=3.3378530564707717e-21)

In [90]:
test_results = ttest_1samp(data_Streets_San["Hourly Rate"],35)
display(test_results.statistic)
display(test_results.pvalue/2)

-9.567447887848152

1.6689265282353859e-21

In [ ]:
#since pvalue is a very close to zero number and it is lower than the significance level,and the t statistical
#falls in the left side, I can reject the null hypothesis 
#We can assess, taking in accoun the data, that hourly wage of hourly workers is less than $35/hour 
# for a 95%confidence interval.

# Challenge 3: To practice - Constructing Confidence Intervals

While testing our hypothesis is a great way to gather empirical evidence for accepting or rejecting the hypothesis, another way to gather evidence is by creating a confidence interval. A confidence interval gives us information about the true mean of the population. So for a 95% confidence interval, we are 95% sure that the mean of the population is within the confidence interval. 
).

To read more about confidence intervals, click [here](https://en.wikipedia.org/wiki/Confidence_interval).


In the cell below, we will construct a 95% confidence interval for the mean hourly wage of all hourly workers. 

The confidence interval is computed in SciPy using the `t.interval` function. You can read more about this function [here](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.t.html).

To compute the confidence interval of the hourly wage, use the 0.95 for the confidence level, number of rows - 1 for degrees of freedom, the mean of the sample for the location parameter and the standard error for the scale. The standard error can be computed using [this](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html) function in SciPy.

In [210]:
# Your code here:

In [229]:
# I extract the relevant sample
data_hourly=data[data["Salary or Hourly"]=="Hourly"]

In [230]:
# I calculate the sample mean
mean_sample=data_hourly["Hourly Rate"].mean()
print (mean_sample)

32.78855771628024


In [231]:
# I calculate the sample std
std_sample=data["Hourly Rate"].std()
print (std_sample)

12.112572684276799


In [232]:
# I calculate the sample size
n=data_hourly["Hourly Rate"].count()
print(n)


8022


In [239]:
# I calculate the mean conficence interval
t_90=t.ppf(0.975,n-1)
confidence_interval_gap=(t_90*(std_sample/(math.sqrt(n))))
print(n)
print(t_90)
print(std_sample)
mean_confidence_interval=[mean_sample-confidence_interval_gap,mean_sample+confidence_interval_gap]
display(mean_confidence_interval)

8022
1.9602597859561675
12.112572684276799


[32.52345834488425, 33.05365708767623]

In [96]:
#same calculation with scipy
sample = data_hourly["Hourly Rate"]
mu=np.mean(sample)
dfr=len(sample)-1
SE=stats.sem(sample)
stats.t.interval(0.95,dfr,loc=mu, scale=SE)

(32.52345834488425, 33.05365708767623)

Now construct the 95% confidence interval for all salaried employeed in the police in the cell below.

In [98]:
# Your code here:

In [99]:
# I extract the relevant sample
data_hourly=data[data["Salary or Hourly"]=="Salary"]

In [100]:
# I calculate the sample mean
mean_sample=data_hourly["Annual Salary"].mean()
print (mean_sample)

86786.99979015143


In [101]:
# I calculate the sample std
std_sample=data["Annual Salary"].std()
print (std_sample)

21041.35460162839


In [102]:
# I calculate the sample size
n=data_hourly["Annual Salary"].count()
print(n)

25161


In [105]:
# I calculate the mean conficence interval
t_90=t.ppf(0.975,n-1)
confidence_interval_gap=(t_90*(std_sample/(math.sqrt(n))))
print(n)
print(t_90)
print(std_sample)
mean_confidence_interval=[mean_sample-confidence_interval_gap,mean_sample+confidence_interval_gap]
display(mean_confidence_interval)

25161
1.9600582764087469
21041.35460162839


[86526.99656774187, 87047.00301256099]

In [104]:
#same calculation with scipy
sample = data_hourly["Annual Salary"]
mu=np.mean(sample)
dfr=len(sample)-1
SE=stats.sem(sample)
stats.t.interval(0.95,dfr,loc=mu, scale=SE)

(86526.99656774187, 87047.00301256099)

# Bonus Challenge - Hypothesis Tests of Proportions

Another type of one sample test is a hypothesis test of proportions. In this test, we examine whether the proportion of a group in our sample is significantly different than a fraction. 

You can read more about one sample proportion tests [here](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/SAS/SAS6-CategoricalData/SAS6-CategoricalData2.html).

In the cell below, use the `proportions_ztest` function from `statsmodels` to perform a hypothesis test that will determine whether the number of hourly workers in the City of Chicago is significantly different from 25% at the 95% confidence level.

In [13]:
# Your code here:

